In [1]:
from IPython.display import IFrame, Image

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "valeria.lupanova Spark RDD app") 

sc = SparkContext(conf=conf)

In [4]:
sc.getConf().getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/data/home/valeria.lupanova/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.11-2.4.5.jar,/data/home/valeria.lupanova/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,/data/home/valeria.lupanova/.ivy2/jars/databricks_spark-sklearn-0.2.3.jar,/data/home/valeria.lupanova/.ivy2/jars/org.apache.kafka_kafka-clients-2.0.0.jar,/data/home/valeria.lupanova/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar,/data/home/valeria.lupanova/.ivy2/jars/org.lz4_lz4-java-1.4.0.jar,/data/home/valeria.lupanova/.ivy2/jars/org.xerial.snappy_snappy-java-1.1.7.3.jar,/data/home/valeria.lupanova/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.history.fs.cleaner.interval', '7d'),
 ('spark.shuffle.io.serverThread

In [6]:
film_id_ = '191'

In [7]:
!hdfs dfs -ls /user/valeria.lupanova/lab06/ml-100k

Found 23 items
-rw-r--r--   2 valeria.lupanova valeria.lupanova       6403 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/README
-rw-r--r--   2 valeria.lupanova valeria.lupanova        716 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/allbut.pl
-rw-r--r--   2 valeria.lupanova valeria.lupanova        643 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/mku.sh
-rw-r--r--   2 valeria.lupanova valeria.lupanova    1979173 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.data
-rw-r--r--   2 valeria.lupanova valeria.lupanova        202 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.genre
-rw-r--r--   2 valeria.lupanova valeria.lupanova         36 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.info
-rw-r--r--   2 valeria.lupanova valeria.lupanova     236344 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.item
-rw-r--r--   2 valeria.lupanova valeria.lupanova        193 2020-11-13 11:21 /user/valeria.lupanova/lab06/ml-100k/u.occupation
-rw-r--

In [8]:
u_item_ = sc.textFile("/user/valeria.lupanova/lab06/ml-100k/u.item")
u_item_

/user/valeria.lupanova/lab06/ml-100k/u.item MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
u_item_.take(5)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0']

In [10]:
u_item_ = u_item_.map(lambda x: x.split("|"))

In [11]:
u_item_.take(5)

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['2',
  'GoldenEye (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?GoldenEye%20(1995)',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0'],
 ['3',
  'Four Rooms (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0'],
 ['4',
  'Get Shorty (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)',
  '0',
  '1',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['5',
  'Copycat 

In [12]:
u_data_ = sc.textFile("/user/valeria.lupanova/lab06/ml-100k/u.data")
u_data_

/user/valeria.lupanova/lab06/ml-100k/u.data MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
u_data_.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [14]:
u_data_ = u_data_.map(lambda x: x.split("\t"))

In [15]:
u_data_.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [16]:
u_data_191_ = u_data_.filter(lambda x: x[1] == film_id_)

In [17]:
u_data_191_.take(3)

[['301', '191', '3', '882075672'],
 ['285', '191', '4', '890595859'],
 ['24', '191', '5', '875323003']]

In [18]:
hist_film_ = u_data_191_.flatMap(lambda x: x[2].split())

In [19]:
hist_film_.collect()

['3',
 '4',
 '5',
 '4',
 '4',
 '4',
 '4',
 '5',
 '5',
 '4',
 '3',
 '3',
 '5',
 '5',
 '4',
 '4',
 '4',
 '4',
 '4',
 '3',
 '4',
 '4',
 '5',
 '4',
 '3',
 '5',
 '5',
 '5',
 '4',
 '5',
 '5',
 '5',
 '5',
 '5',
 '4',
 '5',
 '4',
 '4',
 '3',
 '3',
 '4',
 '4',
 '5',
 '5',
 '5',
 '5',
 '3',
 '4',
 '4',
 '4',
 '3',
 '4',
 '2',
 '5',
 '4',
 '4',
 '5',
 '5',
 '5',
 '5',
 '4',
 '5',
 '4',
 '4',
 '4',
 '4',
 '4',
 '5',
 '4',
 '4',
 '4',
 '5',
 '5',
 '4',
 '3',
 '4',
 '4',
 '4',
 '3',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '5',
 '4',
 '5',
 '5',
 '4',
 '4',
 '5',
 '3',
 '5',
 '5',
 '5',
 '4',
 '4',
 '4',
 '5',
 '5',
 '4',
 '4',
 '4',
 '3',
 '4',
 '5',
 '3',
 '2',
 '5',
 '3',
 '5',
 '4',
 '5',
 '5',
 '5',
 '5',
 '5',
 '2',
 '5',
 '4',
 '3',
 '4',
 '5',
 '4',
 '5',
 '3',
 '4',
 '5',
 '2',
 '4',
 '3',
 '4',
 '3',
 '3',
 '4',
 '5',
 '3',
 '4',
 '4',
 '5',
 '4',
 '4',
 '3',
 '4',
 '5',
 '4',
 '3',
 '4',
 '4',
 '4',
 '5',
 '3',
 '3',
 '5',
 '3',
 '3',
 '5',
 '5',
 '5',
 '5',
 '1',
 '5',
 '5',
 '5',
 '5'

In [20]:
hist_film_.groupBy(lambda x: x).count()

5

In [21]:
hist_film_.take(10)

['3', '4', '5', '4', '4', '4', '4', '5', '5', '4']

In [22]:
type(hist_film_)

pyspark.rdd.PipelinedRDD

In [23]:
hist_film_.count()

276

In [24]:
f_ = hist_film_.countByValue()

In [25]:
hist_film_ = dict(f_)
hist_film_

{'3': 44, '4': 117, '5': 107, '2': 6, '1': 2}

In [26]:
r_hist_film_ = [hist_film_[key] for key in sorted(hist_film_.keys(), reverse=False)]

In [27]:
r_hist_film_

[2, 6, 44, 117, 107]

In [28]:
hist_all_ = u_data_.flatMap(lambda x: x[2].split())

In [29]:
hist_all_.countByKey()

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

In [30]:
h_ = hist_all_.countByValue()

In [31]:
hist_all_ = dict(h_)
hist_all_

{'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201}

In [32]:
r_hist_all_= [hist_all_[key] for key in sorted(hist_all_.keys(), reverse=False)]

In [33]:
r_hist_all_

[6110, 11370, 27145, 34174, 21201]

In [34]:
result_dict_ = {} 
result_dict_["hist_film"] = r_hist_film_
result_dict_["hist_all"] = r_hist_all_
  
print(result_dict_) 

{'hist_film': [2, 6, 44, 117, 107], 'hist_all': [6110, 11370, 27145, 34174, 21201]}


In [35]:
import json

with open('lab06.json', 'w') as fp:
    json.dump(result_dict_, fp)